# Tweets abnehmen und Sentiment analyse anwenden

Die Tweets aus dem Producer werden eingelesen und als PArquet abgelegt.
Darauf wird via *textblob* eine Sentiment analyse gemacht und jedem Tweet ein Rating zugewiesen.

In [0]:
# !pip install textblob

In [0]:
# Libraries einlesen

from pyspark.sql import SparkSession
from pyspark.sql.functions import *   # including udf
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.types import *   
from pyspark.sql import functions as F
from textblob import TextBlob
import random


In [0]:
# nur zum testen: stream sichtbar machen / 

# Create a local/private StreamingContext and SparkContext.
ssc = StreamingContext(sc, 5)   # batch interval = 5
stream = ssc.socketTextStream("localhost", 9997)

# hier die funktionen zum stream in df wandelt einfügen
# dann writestream machen 
stream.pprint()

# Start the computation with timeout function
try:
  ssc.start()                             
  ssc.awaitTerminationOrTimeout(200)  # Ausgabe im consumer erst nach timeout möglich (sekunden)
finally:
  ssc.stop(False)


-------------------------------------------
Time: 2021-05-23 20:55:05
-------------------------------------------

-------------------------------------------
Time: 2021-05-23 20:55:10
-------------------------------------------
RT @dogeofficialceo: Still buying these dips in increments of 69 #doge for good luck.🍀 

God Bless #Dogecoin ❤️🐕

#Bitcoin #btc #xrp #crypt… _t_end_ RT @TakTheApe: Bought myself a new model of Tesla after Elon's energy FUD. 

Pls RT &amp; short everything!
#Lambo #Dogecoin #Moon #Bitcoin #Et… _t_end_ RT @ElonIsWatching: Guys pls vote and share 🤝🌏❤️ #elongate #ELONGATE4CHARITY #ELONGATETOKEN #ELONGATE4CHILDREN #ELONGATEFAMILY https://t.co… _t_end_ @nic__carter This is good for bitcoin. Were gonna need another round of fud dice. _t_end_ RT @maxkeiser: If you are still struggling to understand how #Bitcoin is digital gold, not ‘cash’ read the source code. The verbiage of the… _t_end_ RT @BTC_Archive: Volatility is opportunity. 
#Bitcoin bull market is just getting started! 🙌🚀💥 
https://t.co/LJOSldGrTF _t_end_ @Cereaza2 @ElGrantPratt @TalabaniAsf @om_crypto @elonmusk You don't understand bitcoin at all then and since you sound so sure of yourself I'll let you educate yourself on the implications of that. Or don't. Could care less. _t_end_ RT @bitbns: Bitcoin's price more than 250% higher than a year ago.
...

-------------------------------------------
Time: 2021-05-23 20:55:15
-------------------------------------------
Anyone else… _t_end_ RT @CryptoMichNL: One picture saying it all for #Bitcoin. https://t.co/qKOk4qFHTL _t_end_ @Tambro86 Utility and function you say? Well that rules out Bitcoin 😂👍🏼 _t_end_ RT @OnetLouis: Before the shiba triumph #shiba #SHIBARMY #Bitcoin https://t.co/55L3k81wZD _t_end_ RT @michael_saylor: “There are no tricks in plain and simple faith.” – Julius Caesar on #Bitcoin _t_end_ RT @WuBlockchain: In fact, at present, there is no definite information about China other than the words of the State Council, "crack down… _t_end_ RT @ashwsbreal: Extra 500 #dogecoin to 5 people if I hit 215k today. Just retweet , follow me and tag your friends. 

#cryptocurrencies #Bi… _t_end_ RT @100trillionUSD: Petrodollar is based on the use of highly polluting fossil fuels and needs to be replaced without delay. There is a cle… _t_end_ #trade Crypto Sentiment of Today 🙏 💡Status: Extreme Fear 📊Value: 14/100

#bitcoin  #doge #wink #tipster #bets #crypto #bullish #forex #trade #dogecoin #pump #signal #altcoin https://t.co/yZ8AKIC1bH _t_end_ RT @coinbureau: Yes, #Bitcoin is down from ATH, but its still up over 260% over the year. However, $USD has not fared as favourably. I take… _t_end_ RT @nic__carter: We might be entering the most critical period in bitcoin history, exceeding even the latter half of 2017 _t_end_ Hi users!!
Good news for everyone.
You can stay at home and earn daily $200 Bitcoin in your mobile just simple tap.

Free plan are also available for all users,Don't waste your time and start mining Today!!

...

-------------------------------------------
Time: 2021-05-23 20:55:20
-------------------------------------------
Check out this account on OpenSea https://t.co/3wiFRSVhaH via @opensea _t_end_ @DiMartinoBooth Thank you very much for answering!In my opinion, bitcoin price was just waiting for some news to come down. It was in the Charts before Elon's tweet.Good traders have seen it... Thank you for all your good work and Information. _t_end_ Imagine bitcoin Miami with these shit prices. Everyone would be miserable. 

A crypto conference where no one talks about crypto. 

Now think about a music festival. 
But with no artists and music. 

Shit is going to return. Quit selling. And quit being paper handed pussies. _t_end_ 1/2 Crypto #BTC dump: I am digging a bit deeper into the sell off right now. Guess who is making predictions: BTIG (Rich L.'s club) 😋.
Seems the huge rally of #Bitcoin started after the stopped #GME and #AMC squeeze. Price went all the way up and now down again 🤔... 

In [0]:
# Definiton Tokenizer und Text-preprocess

def preprocessing(lines):
    words = lines.select(explode(split(lines.value, " _t_end_ ")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words


In [0]:
# Definition der Textklassification via Textblob

def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
  
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity


In [0]:
# nur für Test: erzeugen eines Dummy Preises für Bitcoint

def bitcoint_price():
    x = 30000 + random.randrange(100, 1000, 2) 
    return x


In [0]:
# Add sentiment und Preis

def add_newcolumns(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("word"))
    
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
  
    # add the price to the dataframe'''
    add_price_udf = udf(bitcoint_price, StringType())
    words = words.withColumn("price", add_price_udf())  
    return words
  

In [0]:
# Laden der Tweets und Anwendung der textblob_Analyse

# Lokale Spark Session erstellen
spark = SparkSession \
    .builder \
    .appName("TwitterBitcoin") \
    .getOrCreate()

# Tweet data aus dem Socket lesen
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9997) \
    .load()

# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

# Preprocess the data
# words = preprocessing(lines)

# apply text classification
# words = add_newcolumns(words)


In [0]:
words

In [0]:
query = words \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "dbfs:/FileStore/bd_project") \
    .option("checkpointLocation", "./check") \
    .trigger(processingTime='5 seconds') \
    .start()

query.awaitTermination()

In [0]:
# Tweets zusemman mit Sentiment und Preis in ein Parquet Files schreiben (5 sec Fenster)

# words = words.repartition(1)
query = words \
    .writeStream \
    .queryName("all_tweets") \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "dbfs:/FileStore/bd_project") \
    .option("checkpointLocation", "./check") \
    .trigger(processingTime='60 seconds') \
    .start()

query.awaitTermination()
